# Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling1D

# Train

In [2]:
train_set = pd.read_csv('train/train_encoded.csv')

In [3]:
X, y = train_set.iloc[:,:-1],train_set.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Model

In [4]:
batch_size = 1
num_classes = 2
epochs = 10

In [5]:
X_train

,len_keyword,len_location,len_text,len_text_original,diff_len_text,keyword_sum,keyword_min,keyword_max,keyword_mean,keyword_median,...,text_sum_tf-idf,text_min_tf-idf,text_max_tf-idf,text_mean_tf-idf,text_median_tf-idf,text_sum_w2v,text_min_w2v,text_max_w2v,text_mean_w2v,text_median_w2v
1904,5,9,70,105,35,1.000000,1.000000,1.00000,1.000000,1.000000,...,3.592421,0.071013,0.397966,0.299368,0.288652,10.455622,-0.061978,1.000000,0.134046,0.072980
1892,5,2,104,136,32,1.000000,1.000000,1.00000,1.000000,1.000000,...,4.140262,0.198539,0.345046,0.258766,0.242859,36.134873,-0.074673,1.000000,0.142826,0.108250
5640,6,29,72,141,69,1.000000,1.000000,1.00000,1.000000,1.000000,...,3.957329,0.062907,0.599341,0.304410,0.210001,21.251278,-0.135451,1.000000,0.138897,0.099134
850,5,0,42,62,20,1.000000,1.000000,1.00000,1.000000,1.000000,...,2.801855,0.288021,0.465146,0.350232,0.339347,9.890325,-0.019611,0.620719,0.149853,0.122252
1399,8,10,85,136,51,1.000000,1.000000,1.00000,1.000000,1.000000,...,3.758141,0.066550,0.624554,0.289088,0.256348,13.087911,-0.045919,1.000000,0.109066,0.082295
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3203,10,12,63,94,31,1.403003,0.612643,0.79036,0.701501,0.701501,...,2.821905,0.106504,0.473545,0.313545,0.326251,18.172960,-0.007859,1.000000,0.199703,0.147767
6739,12,14,23,29,6,1.000000,1.000000,1.00000,1.000000,1.000000,...,1.730627,0.545210,0.601060,0.576876,0.584356,0.688198,-0.000831,0.331057,0.114700,0.043823
5938,6,17,22,50,28,1.000000,1.000000,1.00000,1.000000,1.000000,...,1.795422,0.120250,0.673899,0.448855,0.500636,1.017684,0.051257,0.516417,0.169614,0.121434
4485,6,12,85,126,41,1.000000,1.000000,1.00000,1.000000,1.000000,...,3.700297,0.075396,0.344611,0.264307,0.270475,13.693841,-0.085002,1.000000,0.114115,0.076175


In [6]:
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)

In [7]:
X_train.shape

(5329, 25, 1)

In [8]:
input_shape = (X_train.shape[1], 1)
input_shape

(25, 1)

## Relu

In [9]:
model = Sequential()
model.add(Conv1D(128, kernel_size=2,
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv1D(64, 1, activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='relu'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 24, 128)           384       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 24, 64)            8256      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 8, 64)             0         
_________________________________________________________________
dropout (Dropout)            (None, 8, 64)             0         
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               65664     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0

In [11]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/5
5329/5329 [==============================] - 15s 3ms/step - loss: 6.6116 - accuracy: 0.5515 - val_loss: 4.3409 - val_accuracy: 0.5679
Epoch 2/5
5329/5329 [==============================] - 17s 3ms/step - loss: 6.6694 - accuracy: 0.5472 - val_loss: 4.4032 - val_accuracy: 0.5679
Epoch 3/5
5329/5329 [==============================] - 16s 3ms/step - loss: 6.5591 - accuracy: 0.5463 - val_loss: 4.5541 - val_accuracy: 0.5679
Epoch 4/5
5329/5329 [==============================] - 16s 3ms/step - loss: 6.5715 - accuracy: 0.5521 - val_loss: 4.7209 - val_accuracy: 0.5679
Epoch 5/5
5329/5329 [==============================] - 19s 4ms/step - loss: 6.6521 - accuracy: 0.5519 - val_loss: 4.9144 - val_accuracy: 0.5679


In [12]:
model = Sequential()
model.add(Conv1D(128, kernel_size=3,
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling1D(pool_size=4))
model.add(Dropout(0.25))
model.add(Dense(128))
model.add(Conv1D(filters=32, kernel_size=1, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(64, 1))
model.add(MaxPooling1D(pool_size=1))
model.add(Flatten())
model.add(Dense(num_classes, activation='relu'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [13]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_2 (Conv1D)            (None, 23, 128)           512       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 5, 128)            0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 5, 128)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 5, 128)            16512     
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 5, 32)             4128      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 2, 32)             0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 2, 64)            

In [14]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/5
5329/5329 [==============================] - 14s 3ms/step - loss: 6.2743 - accuracy: 0.5078 - val_loss: 4.4366 - val_accuracy: 0.4733
Epoch 2/5
5329/5329 [==============================] - 13s 2ms/step - loss: 6.3780 - accuracy: 0.5042 - val_loss: 4.4972 - val_accuracy: 0.4939
Epoch 3/5
5329/5329 [==============================] - 14s 3ms/step - loss: 6.3221 - accuracy: 0.5121 - val_loss: 5.6849 - val_accuracy: 0.5210
Epoch 4/5
5329/5329 [==============================] - 14s 3ms/step - loss: 6.3082 - accuracy: 0.5219 - val_loss: 6.5016 - val_accuracy: 0.5433
Epoch 5/5
5329/5329 [==============================] - 14s 3ms/step - loss: 6.3362 - accuracy: 0.5258 - val_loss: 6.5931 - val_accuracy: 0.5486


In [15]:
model = Sequential()
model.add(Conv1D(1024, kernel_size=1,
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling1D(pool_size=10))
model.add(Dropout(0.25))
model.add(Dense(512))
model.add(Conv1D(filters=32, kernel_size=1, activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(216, 1))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(128, 1))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(64, 1, activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(32, 1, activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Flatten())
model.add(Dense(num_classes, activation='relu'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [16]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_5 (Conv1D)            (None, 25, 1024)          2048      
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 2, 1024)           0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 2, 1024)           0         
_________________________________________________________________
dense_4 (Dense)              (None, 2, 512)            524800    
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 2, 32)             16416     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 2, 32)             0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 2, 216)           

In [17]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/5
5329/5329 [==============================] - 43s 8ms/step - loss: 2.3540 - accuracy: 0.4614 - val_loss: 0.8676 - val_accuracy: 0.5679
Epoch 2/5
5329/5329 [==============================] - 46s 9ms/step - loss: 1.7535 - accuracy: 0.4948 - val_loss: 0.7866 - val_accuracy: 0.5679
Epoch 3/5
5329/5329 [==============================] - 44s 8ms/step - loss: 1.5249 - accuracy: 0.5031 - val_loss: 0.7978 - val_accuracy: 0.5679
Epoch 4/5
5329/5329 [==============================] - 47s 9ms/step - loss: 1.3268 - accuracy: 0.4980 - val_loss: 0.7799 - val_accuracy: 0.5679
Epoch 5/5
5329/5329 [==============================] - 45s 8ms/step - loss: 1.1537 - accuracy: 0.5205 - val_loss: 0.7181 - val_accuracy: 0.5679


## Softmax

In [18]:
model = Sequential()
model.add(Conv1D(128, kernel_size=2,
                 activation='softmax',
                 input_shape=input_shape))
model.add(Conv1D(64, 1, activation='softmax'))
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='softmax'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [19]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/5
5329/5329 [==============================] - 18s 3ms/step - loss: 0.6932 - accuracy: 0.5459 - val_loss: 0.6932 - val_accuracy: 0.5679
Epoch 2/5
5329/5329 [==============================] - 18s 3ms/step - loss: 0.6932 - accuracy: 0.5303 - val_loss: 0.6931 - val_accuracy: 0.5679
Epoch 3/5
5329/5329 [==============================] - 20s 4ms/step - loss: 0.6932 - accuracy: 0.5207 - val_loss: 0.6931 - val_accuracy: 0.5679
Epoch 4/5
5329/5329 [==============================] - 21s 4ms/step - loss: 0.6932 - accuracy: 0.5080 - val_loss: 0.6931 - val_accuracy: 0.5679
Epoch 5/5
5329/5329 [==============================] - 20s 4ms/step - loss: 0.6932 - accuracy: 0.5042 - val_loss: 0.6931 - val_accuracy: 0.5679


## Tanh

In [20]:
model = Sequential()
model.add(Conv1D(128, kernel_size=2,
                 activation='tanh',
                 input_shape=input_shape))
model.add(Conv1D(64, 1, activation='tanh'))
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='tanh'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='tanh'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [21]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/5
5329/5329 [==============================] - 19s 4ms/step - loss: 4.3310 - accuracy: 0.5528 - val_loss: 4.0523 - val_accuracy: 0.5679
Epoch 2/5
5329/5329 [==============================] - 15s 3ms/step - loss: 4.1956 - accuracy: 0.5393 - val_loss: 3.9104 - val_accuracy: 0.5679
Epoch 3/5
5329/5329 [==============================] - 15s 3ms/step - loss: 4.0135 - accuracy: 0.5472 - val_loss: 3.8582 - val_accuracy: 0.5679
Epoch 4/5
5329/5329 [==============================] - 14s 3ms/step - loss: 3.7964 - accuracy: 0.5361 - val_loss: 3.8736 - val_accuracy: 0.5679
Epoch 5/5
5329/5329 [==============================] - 22s 4ms/step - loss: 3.4858 - accuracy: 0.5239 - val_loss: 3.1945 - val_accuracy: 0.5683


# Mix

In [22]:
model = Sequential()
model.add(Conv1D(128, kernel_size=1,
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(0.25))
model.add(Dense(128))
model.add(Conv1D(filters=32, kernel_size=1, activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Conv1D(64, 1))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='relu'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [23]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_15 (Conv1D)           (None, 25, 128)           256       
_________________________________________________________________
max_pooling1d_12 (MaxPooling (None, 8, 128)            0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 8, 128)            0         
_________________________________________________________________
dense_10 (Dense)             (None, 8, 128)            16512     
_________________________________________________________________
conv1d_16 (Conv1D)           (None, 8, 32)             4128      
_________________________________________________________________
max_pooling1d_13 (MaxPooling (None, 2, 32)             0         
_________________________________________________________________
conv1d_17 (Conv1D)           (None, 2, 64)            

In [24]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/5
5329/5329 [==============================] - 17s 3ms/step - loss: 7.4468 - accuracy: 0.4425 - val_loss: 7.6024 - val_accuracy: 0.4321
Epoch 2/5
5329/5329 [==============================] - 16s 3ms/step - loss: 7.4820 - accuracy: 0.4483 - val_loss: 7.6024 - val_accuracy: 0.4321
Epoch 3/5
5329/5329 [==============================] - 18s 3ms/step - loss: 7.3524 - accuracy: 0.4445 - val_loss: 7.6021 - val_accuracy: 0.4321
Epoch 4/5
5329/5329 [==============================] - 14s 3ms/step - loss: 7.3611 - accuracy: 0.4474 - val_loss: 7.5996 - val_accuracy: 0.4321
Epoch 5/5
5329/5329 [==============================] - 17s 3ms/step - loss: 7.2968 - accuracy: 0.4421 - val_loss: 7.5993 - val_accuracy: 0.4321


## Text w2v

In [3]:
train_set = pd.read_csv('train/text_encoded_w2v.csv')

In [4]:
X, y = train_set.iloc[:,:-1],train_set.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Model

In [5]:
batch_size = 1
num_classes = 2
epochs = 10

In [6]:
X_train

,id,text_sum,text_min,text_max,text_mean,text_median
5535,7895,1.885115,-0.031004,0.262091,0.089767,0.059510
6018,8596,9.313427,-0.040400,1.000000,0.102345,0.069088
4739,6738,14.726642,0.012219,1.000000,0.188803,0.132279
3181,4567,3.985746,-0.040585,0.433890,0.110715,0.082433
818,1189,3.576633,-0.090845,0.443895,0.170316,0.168023
...,...,...,...,...,...,...
5282,7548,2.514225,-0.025919,1.000000,0.119725,0.074594
563,814,4.079575,-0.013456,1.000000,0.271972,0.073396
2089,3003,12.973744,-0.108944,1.000000,0.142569,0.114495
3221,4622,3.870319,-0.074206,0.450665,0.107509,0.085664


In [7]:
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)

In [8]:
X_train.shape

(5329, 6, 1)

In [9]:
input_shape = (X_train.shape[1], 1)
input_shape

(6, 1)

## Relu

In [10]:
model = Sequential()
model.add(Conv1D(128, kernel_size=2,
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv1D(64, 1, activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='relu'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 5, 128)            384       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 5, 64)             8256      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 1, 64)             0         
_________________________________________________________________
dropout (Dropout)            (None, 1, 64)             0         
_________________________________________________________________
flatten (Flatten)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 128)               8320      
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0

In [12]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/5
5329/5329 [==============================] - 13s 2ms/step - loss: 8.2134 - accuracy: 0.4460 - val_loss: 8.4215 - val_accuracy: 0.4435
Epoch 2/5
5329/5329 [==============================] - 11s 2ms/step - loss: 8.2596 - accuracy: 0.4466 - val_loss: 8.4191 - val_accuracy: 0.4435
Epoch 3/5
5329/5329 [==============================] - 12s 2ms/step - loss: 8.1505 - accuracy: 0.4399 - val_loss: 8.4215 - val_accuracy: 0.4435
Epoch 4/5
5329/5329 [==============================] - 12s 2ms/step - loss: 8.2660 - accuracy: 0.4384 - val_loss: 8.4215 - val_accuracy: 0.4435
Epoch 5/5
5329/5329 [==============================] - 12s 2ms/step - loss: 8.2680 - accuracy: 0.4372 - val_loss: 8.4216 - val_accuracy: 0.4435


In [14]:
model = Sequential()
model.add(Conv1D(128, kernel_size=1,
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling1D(pool_size=1))
model.add(Dropout(0.25))
model.add(Dense(128))
model.add(Conv1D(filters=32, kernel_size=1, activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(64, 1))
model.add(MaxPooling1D(pool_size=1))
model.add(Flatten())
model.add(Dense(num_classes, activation='relu'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [15]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_4 (Conv1D)            (None, 6, 128)            256       
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 6, 128)            0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 6, 128)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 6, 128)            16512     
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 6, 32)             4128      
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 6, 32)             0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 6, 64)            

In [16]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/5
5329/5329 [==============================] - 14s 3ms/step - loss: 7.9467 - accuracy: 0.5384 - val_loss: 8.3027 - val_accuracy: 0.5565
Epoch 2/5
5329/5329 [==============================] - 13s 2ms/step - loss: 7.9275 - accuracy: 0.5485 - val_loss: 8.2378 - val_accuracy: 0.5565
Epoch 3/5
5329/5329 [==============================] - 14s 3ms/step - loss: 7.8626 - accuracy: 0.5372 - val_loss: 8.1313 - val_accuracy: 0.5565
Epoch 4/5
5329/5329 [==============================] - 13s 2ms/step - loss: 7.8740 - accuracy: 0.5393 - val_loss: 7.7117 - val_accuracy: 0.5565
Epoch 5/5
5329/5329 [==============================] - 13s 2ms/step - loss: 7.8631 - accuracy: 0.5378 - val_loss: 7.4220 - val_accuracy: 0.5565


In [17]:
model = Sequential()
model.add(Conv1D(1024, kernel_size=1,
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling1D(pool_size=1))
model.add(Dropout(0.25))
model.add(Dense(512))
model.add(Conv1D(filters=32, kernel_size=1, activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(216, 1))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(128, 1))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(64, 1, activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(32, 1, activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Flatten())
model.add(Dense(num_classes, activation='relu'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [18]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_7 (Conv1D)            (None, 6, 1024)           2048      
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 6, 1024)           0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 6, 1024)           0         
_________________________________________________________________
dense_5 (Dense)              (None, 6, 512)            524800    
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 6, 32)             16416     
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 6, 32)             0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 6, 216)           

In [19]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/5
5329/5329 [==============================] - 47s 9ms/step - loss: 8.0898 - accuracy: 0.4265 - val_loss: 7.5757 - val_accuracy: 0.4426
Epoch 2/5
5329/5329 [==============================] - 51s 10ms/step - loss: 7.9531 - accuracy: 0.4260 - val_loss: 7.6082 - val_accuracy: 0.4426
Epoch 3/5
5329/5329 [==============================] - 50s 9ms/step - loss: 7.8164 - accuracy: 0.4218 - val_loss: 7.6192 - val_accuracy: 0.4426
Epoch 4/5
5329/5329 [==============================] - 47s 9ms/step - loss: 7.7759 - accuracy: 0.4256 - val_loss: 7.6203 - val_accuracy: 0.4431
Epoch 5/5
5329/5329 [==============================] - 48s 9ms/step - loss: 7.6847 - accuracy: 0.4237 - val_loss: 7.6232 - val_accuracy: 0.4431


## Softmax

In [20]:
model = Sequential()
model.add(Conv1D(128, kernel_size=2,
                 activation='softmax',
                 input_shape=input_shape))
model.add(Conv1D(64, 1, activation='softmax'))
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='softmax'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [21]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/5
5329/5329 [==============================] - 13s 2ms/step - loss: 0.6932 - accuracy: 0.4915 - val_loss: 0.6931 - val_accuracy: 0.4435
Epoch 2/5
5329/5329 [==============================] - 12s 2ms/step - loss: 0.6932 - accuracy: 0.4992 - val_loss: 0.6931 - val_accuracy: 0.4435
Epoch 3/5
5329/5329 [==============================] - 11s 2ms/step - loss: 0.6932 - accuracy: 0.5050 - val_loss: 0.6931 - val_accuracy: 0.4435
Epoch 4/5
5329/5329 [==============================] - 12s 2ms/step - loss: 0.6932 - accuracy: 0.4864 - val_loss: 0.6931 - val_accuracy: 0.4435
Epoch 5/5
5329/5329 [==============================] - 12s 2ms/step - loss: 0.6932 - accuracy: 0.5042 - val_loss: 0.6931 - val_accuracy: 0.4435


## Tanh

In [22]:
model = Sequential()
model.add(Conv1D(128, kernel_size=2,
                 activation='tanh',
                 input_shape=input_shape))
model.add(Conv1D(64, 1, activation='tanh'))
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='tanh'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='tanh'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [23]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/5
5329/5329 [==============================] - 11s 2ms/step - loss: 3.2916 - accuracy: 0.4843 - val_loss: 0.8663 - val_accuracy: 0.4426
Epoch 2/5
5329/5329 [==============================] - 11s 2ms/step - loss: 3.1247 - accuracy: 0.4881 - val_loss: 0.8411 - val_accuracy: 0.4435
Epoch 3/5
5329/5329 [==============================] - 12s 2ms/step - loss: 3.1101 - accuracy: 0.4935 - val_loss: 0.8166 - val_accuracy: 0.4448
Epoch 4/5
5329/5329 [==============================] - 12s 2ms/step - loss: 3.0732 - accuracy: 0.4817 - val_loss: 0.8006 - val_accuracy: 0.4453
Epoch 5/5
5329/5329 [==============================] - 12s 2ms/step - loss: 2.9780 - accuracy: 0.5044 - val_loss: 0.7842 - val_accuracy: 0.4448
